# Adidas Customer Data Churn Analysis

Churn analysis is a technique used to predict or analyze the rate at which customers stop doing business with a company. It is commonly used in industries like telecommunications, subscription services, and banking. Below, I'll walk you through how to perform churn analysis using a sample dataset.

## Step 1: Import necessary libraries and set up the environment

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

##  Step 2: Load and clean the dataset

In [13]:

def create_clean_dataset():
    # Load the dataset
    Adidas_Customer_df = pd.read_csv(r"C:\Users\jki\Desktop\Data Scence Projects\Customer Data Churn Analysis\Machine Learning\Source Files\Adidas US Sales Datasets.csv")
    
    # Print columns to diagnose
    print("Available columns:", Adidas_Customer_df.columns.tolist())
    
    # Convert Invoice_Date to datetime format - fixing column name inconsistency
    if 'Invoice Date' in Adidas_Customer_df.columns:
        Adidas_Customer_df['Invoice_Date'] = pd.to_datetime(Adidas_Customer_df['Invoice Date'])
        # Keep both columns for now to prevent errors
        Adidas_Customer_df['Invoice Date'] = Adidas_Customer_df['Invoice_Date']
    else:
        # If already renamed
        Adidas_Customer_df['Invoice_Date'] = pd.to_datetime(Adidas_Customer_df['Invoice_Date'])
    
    # Calculate Total_Sales if it doesn't exist
    if 'Total Sales' in Adidas_Customer_df.columns:
        Adidas_Customer_df['Total_Sales'] = Adidas_Customer_df['Total Sales']
    elif 'Price' in Adidas_Customer_df.columns and 'Units' in Adidas_Customer_df.columns:
        Adidas_Customer_df['Total_Sales'] = Adidas_Customer_df['Price'] * Adidas_Customer_df['Units']
    elif 'Price' in Adidas_Customer_df.columns and 'Units_Sold' in Adidas_Customer_df.columns:
        Adidas_Customer_df['Total_Sales'] = Adidas_Customer_df['Price'] * Adidas_Customer_df['Units_Sold']
    
    # Sort by date
    Adidas_Customer_df = Adidas_Customer_df.sort_values('Invoice_Date')
    
    # Add a Year_Month column for time-based analysis
    Adidas_Customer_df['Year_Month'] = Adidas_Customer_df['Invoice_Date'].dt.to_period('M')
    
    return Adidas_Customer_df


# Check the data type status
Adidas_Customer_df = create_clean_dataset()
Adidas_Customer_df.info()

   

Available columns: ['Customer Name', 'Customer ID', 'Invoice Date', 'Customer Location', 'Customer State', 'Customer City', 'Product', 'Price per Unit', 'Units Sold', 'Total Sales', 'Sales Method']
<class 'pandas.core.frame.DataFrame'>
Index: 9648 entries, 0 to 1200
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Customer Name      9648 non-null   object        
 1   Customer ID        9648 non-null   int64         
 2   Invoice Date       9648 non-null   datetime64[ns]
 3   Customer Location  9648 non-null   object        
 4   Customer State     9648 non-null   object        
 5   Customer City      9648 non-null   object        
 6   Product            9648 non-null   object        
 7   Price per Unit     9648 non-null   object        
 8   Units Sold         9648 non-null   object        
 9   Total Sales        9648 non-null   object        
 10  Sales Method       9648 non-null   

In [16]:
# Check the data type status
Adidas_Customer_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9648 entries, 0 to 1200
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Customer Name      9648 non-null   object        
 1   Customer ID        9648 non-null   int64         
 2   Invoice Date       9648 non-null   datetime64[ns]
 3   Customer Location  9648 non-null   object        
 4   Customer State     9648 non-null   object        
 5   Customer City      9648 non-null   object        
 6   Product            9648 non-null   object        
 7   Price per Unit     9648 non-null   object        
 8   Units Sold         9648 non-null   object        
 9   Total Sales        9648 non-null   object        
 10  Sales Method       9648 non-null   object        
 11  Invoice_Date       9648 non-null   datetime64[ns]
 12  Total_Sales        9648 non-null   object        
 13  Year_Month         9648 non-null   period[M]     
dtypes: datetime64

## Step 3: Analyze customer churn

In [32]:
import pandas as pd

def analyze_customer_churn(Adidas_Customer_df):
    """
    Analyzes customer churn based on monthly customer activity.

    Args:
        Adidas_Customer_df (pd.DataFrame): DataFrame containing customer transaction data.
                                            Must have columns 'Year_Month' and 'Customer Name'.

    Returns:
        tuple: A tuple containing:
            - churn_analysis (pd.DataFrame): DataFrame with churn metrics per month.
            - customer_activity (pd.DataFrame): DataFrame showing customer activity over time.
    """

    # Check if required columns exist
    if not all(col in Adidas_Customer_df.columns for col in ['Year_Month', 'Customer Name']):
        raise ValueError("Input DataFrame must contain 'Year_Month' and 'Customer Name' columns.")

    # Ensure 'Year_Month' is sorted
    Adidas_Customer_df = Adidas_Customer_df.sort_values(by='Year_Month')

    # Count unique customers per month
    customer_counts = Adidas_Customer_df.groupby('Year_Month')['Customer Name'].nunique()

    # Track customer activity over time
    customer_activity = Adidas_Customer_df.groupby(['Year_Month', 'Customer Name']).size().unstack(fill_value=0)

    # Convert to binary activity (1 = active, 0 = inactive)
    customer_activity_binary = customer_activity.applymap(lambda x: 1 if x > 0 else 0)

    # Calculate churn: customers who were active in previous month but not in current month
    churn_analysis = pd.DataFrame(index=customer_activity_binary.index[1:], columns=['Active_Customers', 'Churned_Customers', 'Churn_Rate'])

    for i in range(1, len(customer_activity_binary)):
        prev_month = customer_activity_binary.index[i-1]
        curr_month = customer_activity_binary.index[i]

        # Customers active in previous month
        prev_active = set(customer_activity_binary.columns[customer_activity_binary.iloc[i-1] == 1])

        # Customers active in current month
        curr_active = set(customer_activity_binary.columns[customer_activity_binary.iloc[i] == 1])

        # Customers who churned (were active in previous month but not in current month)
        churned = prev_active - curr_active

        # Calculate churn rate
        if len(prev_active) > 0:
            churn_rate = len(churned) / len(prev_active) * 100
        else:
            churn_rate = 0

        churn_analysis.loc[curr_month, 'Active_Customers'] = len(curr_active)
        churn_analysis.loc[curr_month, 'Churned_Customers'] = len(churned)
        churn_analysis.loc[curr_month, 'Churn_Rate'] = churn_rate

    return churn_analysis, customer_activity
  rn churn_analysis, customer_activity

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 60)

In [33]:
return churn_analysis, customer_activity

SyntaxError: 'return' outside function (159834887.py, line 1)

In [ ]:
## Step 4: Analyze sales patterns

In [ ]:

def analyze_sales_patterns(df):
    # Check if Total_Sales column exists, if not create it
    if 'Total_Sales' not in df.columns:
        # Assuming there might be 'Price' and 'Units_Sold' columns
        if 'Price' in df.columns and 'Units_Sold' in df.columns:
            df['Total_Sales'] = df['Price'] * df['Units_Sold']
        # Fallback if there's a 'Total Sales' column with space
        elif 'Total Sales' in df.columns:
            df = df.rename(columns={'Total Sales': 'Total_Sales'})
    
    # Analyze sales trends by customer
    customer_sales = df.groupby(['Customer Name', 'Year_Month'])['Total_Sales'].sum().unstack(fill_value=0)
    
    # Product popularity by customer
    # Check if 'Product' column exists, use alternative if needed
    product_col = 'Product' if 'Product' in df.columns else 'Product Name' if 'Product Name' in df.columns else None
    if product_col:
        product_preference = df.groupby(['Customer Name', product_col])['Total_Sales'].sum().unstack(fill_value=0)
    else:
        product_preference = pd.DataFrame()  # Empty dataframe if no product column found
    
    # Sales method preference
    # Check if 'Sales_Method' column exists, use alternative if needed
    sales_method_col = 'Sales_Method' if 'Sales_Method' in df.columns else 'Sales Method' if 'Sales Method' in df.columns else None
    if sales_method_col:
        sales_method_preference = df.groupby(['Customer Name', sales_method_col])['Total_Sales'].sum().unstack(fill_value=0)
    else:
        sales_method_preference = pd.DataFrame()  # Empty dataframe if no sales method column found
    
    return customer_sales, product_preference, sales_method_preference

## Step 5: Perform customer segmentation

In [ ]:
def customer_segmentation(df):
    # Create customer summary
    customer_summary = df.groupby('Customer Name').agg({
        'Invoice_Date': [min, max, 'count'],
        'Total_Sales': 'sum'
    })
    
    customer_summary.columns = ['First_Purchase', 'Last_Purchase', 'Purchase_Count', 'Total_Revenue']
    
    # Add Units_Sold if available
    if 'Units_Sold' in df.columns:
        units_sold = df.groupby('Customer Name')['Units_Sold'].sum()
        customer_summary['Total_Units'] = units_sold
    
    # Calculate days since last purchase
    latest_date = df['Invoice_Date'].max()
    customer_summary['Days_Since_Last_Purchase'] = (latest_date - customer_summary['Last_Purchase']).dt.days
    
    # Calculate average purchase value
    customer_summary['Avg_Purchase_Value'] = customer_summary['Total_Revenue'] / customer_summary['Purchase_Count']
    
    # Calculate customer lifetime in days
    customer_summary['Customer_Lifetime'] = (customer_summary['Last_Purchase'] - customer_summary['First_Purchase']).dt.days
    
    # Calculate purchase frequency (purchases per month)
    customer_summary['Purchase_Frequency'] = customer_summary['Purchase_Count'] / ((customer_summary['Customer_Lifetime'] / 30) + 1)  # Add 1 to avoid division by zero
    
    return customer_summary
        

# Step 6: Calculate retention rate over time

In [ ]:
def calculate_retention(Adidas_Customer_df):
    months = sorted(Adidas_Customer_df['Year_Month'].unique())
    retention_data = []

    for i in range(1, len(months)):
        curr_month = months[i]
        prev_month = months[i-1]
        
        # Get active customers in both months - fixing column name inconsistency
        prev_customers = set(Adidas_Customer_df[Adidas_Customer_df['Year_Month'] == prev_month]['Customer Name'].unique())
        curr_customers = set(Adidas_Customer_df[Adidas_Customer_df['Year_Month'] == curr_month]['Customer Name'].unique())
        
        # Calculate retained customers
        retained = prev_customers.intersection(curr_customers)
        retention_rate = len(retained) / len(prev_customers) * 100 if len(prev_customers) > 0 else 0
        
        retention_data.append({
            'Month': curr_month,
            'Retention_Rate': retention_rate,
            'Active_Customers': len(curr_customers),
            'Retained_Customers': len(retained)
        })

    retention_df = pd.DataFrame(retention_data)
    return retention_df



Visualize Customer Churn

We can plot the churn rate over time to see how it changes month by month.

In [ ]:
def visualize_churn(churn_analysis):
    plt.figure(figsize=(12, 6))
    plt.plot(churn_analysis.index.astype(str), churn_analysis['Churn_Rate'], marker='o', linestyle='-', color='b')
    plt.title('Monthly Customer Churn Rate')
    plt.xlabel('Month')
    plt.ylabel('Churn Rate (%)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

Visualize Sales Patterns


We can visualize sales trends over time and product preferences

In [ ]:
def visualize_sales_patterns(customer_sales, product_preference):
    # Sales trends over time
    plt.figure(figsize=(12, 6))
    customer_sales.sum().plot(kind='line', marker='o', linestyle='-', color='g')
    plt.title('Total Sales Over Time')
    plt.xlabel('Month')
    plt.ylabel('Total Sales')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

    # Product preferences
    if not product_preference.empty:
        plt.figure(figsize=(12, 6))
        product_preference.sum().sort_values(ascending=False).plot(kind='bar', color='orange')
        plt.title('Product Popularity')
        plt.xlabel('Product')
        plt.ylabel('Total Sales')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.show()

Visualize Customer Segmentation


We can visualize customer segments based on their purchase behavior.

In [ ]:
def visualize_customer_segments(customer_segments):
    # Days since last purchase
    plt.figure(figsize=(12, 6))
    sns.histplot(customer_segments['Days_Since_Last_Purchase'], bins=30, kde=True, color='purple')
    plt.title('Distribution of Days Since Last Purchase')
    plt.xlabel('Days Since Last Purchase')
    plt.ylabel('Number of Customers')
    plt.grid(True)
    plt.show()

    # Purchase frequency
    plt.figure(figsize=(12, 6))
    sns.histplot(customer_segments['Purchase_Frequency'], bins=30, kde=True, color='teal')
    plt.title('Distribution of Purchase Frequency')
    plt.xlabel('Purchase Frequency (Purchases per Month)')
    plt.ylabel('Number of Customers')
    plt.grid(True)
    plt.show()

Visualize Retention Rates


We can plot the retention rate over time to see how customer retention changes.on


In [ ]:
def visualize_retention(retention_df):
    plt.figure(figsize=(12, 6))
    plt.plot(retention_df['Month'].astype(str), retention_df['Retention_Rate'], marker='o', linestyle='-', color='r')
    plt.title('Monthly Customer Retention Rate')
    plt.xlabel('Month')
    plt.ylabel('Retention Rate (%)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

Visualize At-Risk Customers


We can visualize the customers who are at risk of churning based on their inactivity

In [ ]:
def visualize_at_risk_customers(at_risk):
    plt.figure(figsize=(12, 6))
    sns.histplot(at_risk['Days_Since_Last_Purchase'], bins=30, kde=True, color='red')
    plt.title('Distribution of Days Since Last Purchase for At-Risk Customers')
    plt.xlabel('Days Since Last Purchase')
    plt.ylabel('Number of Customers')
    plt.grid(True)
    plt.show()

Main Function to Run Visualizations


We can integrate all the visualization functions into the main analysis function.

In [ ]:
def run_visualizations(results):
    # Visualize churn analysis
    visualize_churn(results['churn_analysis'])
    
    # Visualize sales patterns
    visualize_sales_patterns(results['customer_sales'], results['product_preference'])
    
    # Visualize customer segments
    visualize_customer_segments(results['customer_segments'])
    
    # Visualize retention rates
    visualize_retention(results['retention_df'])
    
    # Visualize at-risk customers
    visualize_at_risk_customers(results['at_risk'])

Execute the Analysis and Visualizations


Finally, we can run the analysis and generate the visualizations.

In [ ]:
if __name__ == "__main__":
    # Run the churn analysis
    results = run_churn_analysis()
    
    # Generate visualizations
    run_visualizations(results)

Summary of Visualizations:
Churn Rate Over Time: A line plot showing the monthly churn rate.

Sales Trends Over Time: A line plot showing total sales over time.

Product Popularity: A bar chart showing the most popular products.

Customer Segmentation:

Distribution of days since the last purchase.

Distribution of purchase frequency.

Retention Rate Over Time: A line plot showing the monthly retention rate.

At-Risk Customers: A histogram showing the distribution of days since the last purchase for at-risk customers.